### Setting up the GPU for the Deep Learning

In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [2]:
# #above environment variable set up cab be done another way as well
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=1

In [3]:
import tensorflow as tf
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [4]:
#tf v1 version
# import tensorflow as tf
# tf.config.gpu.set_per_process_memory_fraction(0.75)
# tf.config.gpu.set_per_process_memory_growth(True)

In [5]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [6]:
from keras.datasets import mnist,cifar10

Using TensorFlow backend.


In [7]:
from keras.utils import to_categorical
from PIL import Image

In [8]:
def load_cifar10(resize=False):
    train = np.load('cifar10_images/cifar_train.npz')
    x_train = train['data']
    y_train = train['labels']
    
    test = np.load('cifar10_images/cifar_test.npz')
    x_test = test['data']
    y_test = test['labels']
    
    if resize:
        x_train=resize_all(x_train, resize)
        x_test=resize_all(x_test, resize)
    
    x_train=x_train.astype('float32')/255
    x_test=x_test.astype('float32')/255

    return x_train,x_test,y_train,y_test

In [9]:
def resize(p, size):
    return Image.fromarray(p).resize(size=(size,size))

def resize_all(arr, size):
    t = []
    for i in range(arr.shape[0]):
        t.append(np.array(resize(arr[i], size)))

In [10]:
batch_size = 64
nb_classes = 10 #number classes

img_rows, img_cols = 32, 32    # input image dimensions
img_channels = 3  

In [96]:
#loading the data
(x_mntrain, y_mntrain), (x_mntest, y_mntest) = mnist.load_data()
(x_cfartrain, y_cfartrain), (x_cfartest, y_cfartest)=cifar10.load_data()

In [97]:
x_mntrain = x_mntrain.astype('float32')/255.
x_mntest = x_mntest.astype('float32')/255.
x_cfartrain = x_cfartrain.astype('float32')/255.
x_cfartest = x_cfartest.astype('float32')/255.

In [98]:
print(x_mntrain.shape)
print(x_mntest.shape)
print(x_cfartrain.shape)
print(x_cfartest.shape)

(60000, 28, 28)
(10000, 28, 28)
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [99]:
x_mntrain=x_mntrain.reshape(-1,28,28,1)
x_mntest=x_mntest.reshape(-1,28,28,1)

In [100]:
print(x_mntrain.shape)
print(x_mntest.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [101]:
print(y_mntrain.shape)
print(y_mntest.shape)
print(y_cfartrain.shape)
print(y_cfartest.shape)

(60000,)
(10000,)
(50000, 1)
(10000, 1)


In [103]:
y_mntrain = to_categorical(y_mntrain, nb_classes)
y_mntest = to_categorical(y_mntest, nb_classes)
y_cfartrain = to_categorical(y_cfartrain, nb_classes)
y_cfartest = to_categorical(y_cfartest, nb_classes)

In [104]:
print(y_mntrain.shape)
print(y_mntest.shape)
print(y_cfartrain.shape)
print(y_cfartest.shape)

(60000, 10)
(10000, 10)
(50000, 10)
(10000, 10)


In [26]:
from keras import Sequential
from keras.layers import Dense, Convolution2D, Convolution3D,Dropout, BatchNormalization, Flatten,MaxPool2D
from keras.optimizers import adam,RMSprop
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [106]:
model=Sequential()
model.add(Convolution2D(32,kernel_size=(4,4),padding='same',activation='relu',input_shape=(28,28,1)))
model.add(Convolution2D(32,kernel_size=(4,4),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,kernel_size=(4,4),padding='same',activation='relu'))
model.add(Convolution2D(64,kernel_size=(4,4),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [139]:
optimizor=RMSprop(learning_rate=0.001,rho=0.9, epsilon=1e-08, decay=0.0)
ada=adam()

In [115]:
model.compile(optimizer=optimizor,loss='categorical_crossentropy',metrics=['accuracy'])

In [118]:
reduceCallback = ReduceLROnPlateau(monitor='val_acc',  patience=3, verbose=1,  factor=0.5, min_lr=0.00001)

In [124]:
earlystop=EarlyStopping()

In [121]:
#data without augmentaion
model.fit(x_mntrain,y_mntrain,batch_size=30,epochs=2,validation_data=(x_mntest,y_mntest),callbacks=[earlystop,reduceCallback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 178s 3ms/step - loss: 0.0712 - accuracy: 0.9826 - val_loss: 0.0282 - val_accuracy: 0.9910
Epoch 2/2
60000/60000 [==============================] - 178s 3ms/step - loss: 0.0770 - accuracy: 0.9819 - val_loss: 0.0419 - val_accuracy: 0.9898


In [122]:
imageAug=ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=F,rotation_range=15,zoom_range=0.1,width_shift_range=0.1)

In [123]:
imageAug.fit(x_mntrain)

In [132]:
model.fit_generator(imageAug.flow(x_mntest, y_mntest, batch_size=82),epochs=30,validation_data=(x_mntest,y_mntest),callbacks=[reduceCallback])

Epoch 1/30
122/122 [==============================] - 22s 182ms/step - loss: 0.1505 - accuracy: 0.9666 - val_loss: 2.2476 - val_accuracy: 0.1216
Epoch 2/30
122/122 [==============================] - 23s 187ms/step - loss: 0.1320 - accuracy: 0.9709 - val_loss: 2.2847 - val_accuracy: 0.1136
Epoch 3/30
122/122 [==============================] - 23s 191ms/step - loss: 0.1033 - accuracy: 0.9732 - val_loss: 2.2629 - val_accuracy: 0.1136
Epoch 4/30
122/122 [==============================] - 23s 186ms/step - loss: 0.1090 - accuracy: 0.9721 - val_loss: 2.2938 - val_accuracy: 0.1135
Epoch 5/30
122/122 [==============================] - 23s 188ms/step - loss: 0.1072 - accuracy: 0.9771 - val_loss: 2.2448 - val_accuracy: 0.1152
Epoch 6/30
122/122 [==============================] - 23s 187ms/step - loss: 0.1088 - accuracy: 0.9749 - val_loss: 2.3591 - val_accuracy: 0.1136
Epoch 7/30
122/122 [==============================] - 23s 187ms/step - loss: 0.1152 - accuracy: 0.9760 - val_loss: 2.1686 - val_ac

In [136]:
model1=Sequential()
model1.add(Convolution2D(32,kernel_size=(4,4),padding='same',activation='relu',input_shape=(32,32,3)))
model1.add(Convolution2D(32,kernel_size=(4,4),padding='same',activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))

model1.add(Convolution2D(64,kernel_size=(4,4),padding='same',activation='relu'))
model1.add(Convolution2D(64,kernel_size=(4,4),padding='same',activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))

model1.add(Convolution2D(128,kernel_size=(4,4),padding='same',activation='relu'))
model1.add(Convolution2D(128,kernel_size=(4,4),padding='same',activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(256,activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(10,activation='softmax'))

In [137]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_cfartrain)

In [141]:
model1.compile(optimizer=ada,loss='categorical_crossentropy',metrics=['accuracy'])

In [142]:
# fits the model on batches with real-time data augmentation:
model1.fit_generator(datagen.flow(x_cfartrain, y_cfartrain, batch_size=32),
                    steps_per_epoch=len(x_cfartrain) / 32, epochs=5,callbacks=[reduceCallback])

Epoch 1/5
1563/1562 [==============================] - 218s 140ms/step - loss: 1.8367 - accuracy: 0.3057
Epoch 2/5
1563/1562 [==============================] - 222s 142ms/step - loss: 1.5380 - accuracy: 0.4334
Epoch 3/5
1563/1562 [==============================] - 224s 143ms/step - loss: 1.4312 - accuracy: 0.4813
Epoch 4/5
1563/1562 [==============================] - 224s 144ms/step - loss: 1.3681 - accuracy: 0.5086
Epoch 5/5
1563/1562 [==============================] - 224s 144ms/step - loss: 1.3194 - accuracy: 0.5310
